In [1]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS
import fig_plot as fp

In [2]:
N_sample = 20
seed = 1
solver_type = 'minlp'
model_type = 'deterministic'
data = ATCS(seed = 1)
data.choose_subset_point(N_sample) # Choose subset data

In [3]:
# location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
# location_fig.show()

In [4]:
data.r_sub_df.head()

,range
index,
120,127.887414
552,60.391929
90,130.136422
973,65.957089
667,41.008548


In [5]:
# MINLP Model
# Data
robot_id = data.r_sub_df.index.to_numpy()
robot_loc = data.l_sub_df.to_numpy()
robot_range = data.r_sub_df.to_numpy().flatten()
dist_matrix = data.get_distance_matrix(sample_subset = True)

# Creates an empty Model
model = xp.problem(name='MINLP_Deterministic')

# Given Inputs and Parameters
R = data.r_sub_df['range'].to_dict()
L_Range = data.r_min # km
U_Range = data.r_max # km
U_Robot = data.q # Max robots per charger
U_Charger = data.m # Max chargers per station
L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station

C_b = data.c_b # Investment cost per station
C_h = data.c_h # Cost of moving a robot
C_m = data.c_m # Maintenance cost per charger
C_c = data.c_c # Charging cost per km


Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()

# Parameters
U_Station = L_Station + 2 # For now

data.set_output_folder(solver_type, model_type, name = f'{N_sample}_samp_{U_Station}s_seed{seed}')

# # U_Station = N_sample
# # Sets and Notation
# V = data.r_sub_df.index.to_numpy() # Robot Vehicle sets
# S = np.arange(U_Station)


# # Output Variables
# x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = -xp.infinity) # Longitude of station s
# y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = -xp.infinity) # Latitude of station s

# eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
# mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
# beta = model.addVariables(V, S, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
# alpha = model.addVariables(V, S, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
# d = model.addVariables(V, S, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


# # Constraints
# # Robot v can be allocated to charging station s iff it is built
# model.addConstraint(beta[v,s] <= mu[s] for v in V for s in S) 

# # Each Robot v must be allocated to only one station
# model.addConstraint(xp.Sum(beta[v,s] for s in S) == 1 for v in V)

# # Number of chargers per station
# model.addConstraint(eta[s] >= xp.Sum(beta[v,s] for v in V)/U_Robot for s in S)
# model.addConstraint(eta[s] <= U_Charger for s in S)
# # Robot v can be brought to station s by human iff the robot is allocated to that station
# model.addConstraint(alpha[v,s] <= beta[v,s] for v in V for s in S)

# # Distance from robot v to station s
# # If Robot v is allocated to station s,
# #       If its range can reach s, alpha = 0
# #       Else; distance is 0 and it is brought to s by taking penalty (alpha = 1)
# # Else; d[v,s] = 0
# model.addConstraint(d[v,s] <= R[v] * (1 - alpha[v,s]) for v in V for s in S)
# model.addConstraint(d[v,s] >= xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
#                                 - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
#                                         for v in V for s in S)
# # model.addConstraint(d[v,s]**2 >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
# #                                 - (np.max(dist_matrix)**2) * (alpha[v,s] + (1 - beta[v,s])) 
# #                                         for v in V for s in S)
# # model.addConstraint(d[v,s] >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
# #                                 - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
# #                                         for v in V for s in S)
# model.addConstraint(d[v,s] <= np.max(dist_matrix) * beta[v,s] for v in V for s in S)
# # model.addConstraint(alpha[120,0] == 1)
# # model.addConstraint(d[v,s] <= np.min(dist_matrix[dist_matrix != 0]) * (1 - alpha[v,s]) for v in V for s in S)
# # model.addConstraint(d[v,s] >= np.min(dist_matrix[dist_matrix != 0]) - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
# #                                     for v in V for s in S)
# # model.addConstraint(d[v,s] <= np.max(dist_matrix)*beta[v,s] for v in V for s in S)

# # Objective Function
# Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
# Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)
# Penalty_cost = xp.Sum(C_h * alpha[v,s] for v in V for s in S)
# Charging_cost = xp.Sum(C_c * (beta[v,s]*U_Range - (beta[v,s]*R[v] - d[v,s])) for v in V for s in S)

# obj = Build_station_cost + Build_charger_cost + Charging_cost + Penalty_cost

# model.setObjective(obj, sense=xp.minimize)
# model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', 3600)
# # Solve the problem
# model.solve()

In [6]:
np.min(dist_matrix[dist_matrix != 0])

2.0502244607013913

In [7]:
model.getSolution(obj)

NameError: name 'obj' is not defined

In [ ]:
mu_dict = model.getSolution(mu)
eta_dict = model.getSolution(eta)
x_dict = model.getSolution(x)
y_dict = model.getSolution(y)

station_df = pd.DataFrame([mu_dict, eta_dict, x_dict, y_dict], ['mu','eta','longitude','latitude']).T
station_df['Building_Station_Cost'] = station_df['mu'] * C_b
station_df['Building_Charger_Cost'] = station_df['eta'] * C_m

In [ ]:
beta_dict = model.getSolution(beta)
alpha_dict = model.getSolution(alpha)
d_dict = model.getSolution(d)

robot_df = pd.DataFrame([beta_dict, alpha_dict, d_dict],['beta','alpha','d']).T.reset_index()
robot_df.rename(columns={'index':'(v,s)'}, inplace=True)
robot_df['v'], robot_df['s'] = robot_df['(v,s)'].apply(lambda x: x[0]), robot_df['(v,s)'].apply(lambda x: x[1])
robot_df.drop(['(v,s)'], axis = 1, inplace= True)
robot_df = robot_df[['v','s','beta','alpha','d']]
# robot_df = robot_df.groupby(['s','v']).sum()

In [ ]:
model.getSolution(Charging_cost)

1009.1452275994271

In [ ]:
robot_df[robot_df['beta']==1]

,v,s,beta,alpha,d
0,120,0,1.0,0.0,44.114092
5,552,1,1.0,0.0,17.832107
9,90,1,1.0,1.0,0.000000
12,973,0,1.0,0.0,9.532157
17,667,1,1.0,0.0,3.525094
20,314,0,1.0,0.0,50.515308
24,181,0,1.0,1.0,0.000000
28,119,0,1.0,0.0,79.474763
32,49,0,1.0,1.0,0.000000
36,101,0,1.0,1.0,0.000000


In [ ]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,1.0,7.0,-65.557092,-72.256467,5000.0,3500.0
1,1.0,3.0,148.908637,-83.688787,5000.0,1500.0
2,0.0,0.0,-152.253916,-84.543611,0.0,0.0
3,0.0,0.0,-152.222153,-89.787140,0.0,0.0


In [ ]:
robot_df.to_csv('minlp_output/20_samp_4s/robot_df.csv')
station_df.to_csv('minlp_output/20_samp_4s/station_df.csv')

In [ ]:
heu_station_df = pd.DataFrame( [(-120.28422860594455, -85.14822308962202), (-43.75665270724203, -70.98962125892272), (150.21514729388687, -83.492662445044)])
heu_station_df.columns = ['longitude','latitude']

In [ ]:
heu_station_df

,longitude,latitude
0,-120.284229,-85.148223
1,-43.756653,-70.989621
2,150.215147,-83.492662


In [ ]:
location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig = fp.add_point_to_l_fig(location_fig, station_df[station_df['mu'] != 0], size=7, name='MINLP Stations', color='green')
location_fig = fp.add_point_to_l_fig(location_fig, heu_station_df, size=7, name='Heuristic Stations', color='Brown')
location_fig.show()